In [132]:
import pandas as pd
# from fuzzywuzzy import process,fuzz
from tqdm import tqdm,trange
import re
import sqldf

In [ ]:
# # fuzzywuzzy:
# # wont crash but take forever to run:
# def match_companies(l1, l2, threshold=70):
#     l2,l1=max(l1,l2,key=lambda x:len(x)),min(l1,l2,key=lambda x:len(x))
#     matches = []
#     for company in tqdm(l1):
#         best_match = process.extractOne(company, l2, score_cutoff=threshold)
#         if best_match:
#             matches.append((company, best_match[0]))
#     return matches
# factMatch = match_companies(bwFacts,ciFacts)

# # word embedding
# from transformers import AutoTokenizer, AutoModel
# import torch
# from sklearn.metrics.pairwise import cosine_similarity
# def get_embeddings(texts):
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state[:, 0, :].numpy()
# # running this 2 lines below will cause crashing: 
# bwE=get_embeddings(bwFacts)
# ciE=get_embeddings(ciFacts)
# similarity_matrix = cosine_similarity(embeddings_l1, embeddings_l2)

In [3]:
# read and preprocessing:
bw=pd.read_excel("QuestID_FactoryName.xls",sheet_name="data")
ci=pd.read_csv("V2_COMPANY_IDENTIFIERS.csv",encoding="latin-1").rename({"ï»¿reprisk_id":"repriskID"},axis=1)
ci=ci[ci["headquarters_country"].isin(["Bangladesh","Cambodia","Indonesia","Haiti","Nicaragua","Jordan","Viet Nam (Vietnam)"])]
ci.loc[ci["headquarters_country"]=="Viet Nam (Vietnam)", "headquarters_country"]="Vietnam"

In [162]:
re.sub(r'co\.?\,?\s?ltd\.?',"","co ltd")

''

|BW|CompanyId|
|---|---|
|x formerly(y)|y|
|x formerly(y)|x|

In [88]:
def strProcess(s:str, aggresive=False):
    s=s.replace("Co.,","Co.")
    s=s.lower()
    s=s.replace("&"," and ").replace("–","-")
    s=s.replace("mfg.","manufacturing").replace("ltd co","")
    if "former" not in s:
        s=re.sub(r'\(.*?\)$', '', s)
    s=re.sub(r'\s{2,}', ' ', s)
    if aggresive==True:
        s=re.sub(r'-\s.{2,}',"",s)
        s=re.sub(r'co\.?\,?\s?ltd\.?',"",s)
    s=re.sub(r'[^\w\d]*$',"",s)
    return s

def getBWDuplicates(verbose=False, col="FactoryName"):
    nameCounts=bw.groupby("FactoryBWID")[col].nunique()
    susNames=nameCounts[nameCounts > 1].index
    opdfv=bw[bw['FactoryBWID'].isin(susNames)]
    opdf=opdfv.groupby("FactoryBWID")[col].unique().apply(list).to_dict()
    if verbose==False:
        return opdf
    else:
        return opdfv

In [81]:
bw.groupby("FactoryBWID")["FactoryName"].nunique()

FactoryBWID
2       1
4       1
5       2
6       2
7       1
       ..
3722    1
3778    1
3788    1
3796    1
3820    1
Name: FactoryName, Length: 1982, dtype: int64

In [147]:
[x for x in ciFacts if "Cambodia" in x]

['Canteran Apparel (Cambodia) Co Ltd',
 'iCare Benefits (Cambodia) Co Ltd',
 'First and Main Cambodia Ltd',
 'Cambodia International Investment Development Group Co Ltd',
 'Horizon Outdoor Cambodia Co Ltd',
 'Kingsland Garment (Cambodia) Ltd',
 'Gladpeer Garments Factory (Cambodia) Ltd',
 'Mong Reththy Investment Cambodia Oil Palm Co Ltd',
 'CJ Cambodia Co Ltd',
 '(Cambodia) Tong Min Group Engineering Co Ltd',
 'Cambodia Iron and Steel Mining Industry Group Ltd (CISMIG)',
 'SL Garment Processing Cambodia Ltd',
 'Daun Penh (Cambodia) Group Co Ltd',
 'Top World Garment (Cambodia) Ltd',
 'Sabrina (Cambodia) Garment Manufacturing Corp',
 'Kingdom Breweries Cambodia Ltd',
 'Cambodia Hydropower Development (CHD)',
 'Cambodian Power Grid Co Ltd',
 'ANZ Royal Bank Cambodia Ltd',
 'Clik Payment (Cambodia) Co., Ltd.',
 'Cambodian Television Network (CTN)',
 'Philimore Cambodia Co Ltd',
 'Lan Feng (Cambodia) International Co Ltd',
 'Daqian Textile (Cambodia) Co Ltd',
 'New Wide (Cambodia) Garment

In [10]:
bwFacts=list(bw.FactoryName.unique()) #len: 2094
ciFacts=list(ci.company_name.unique()) #len: 15339

In [76]:
bw["FNC"]=bw["FactoryName"].apply(strProcess, aggresive=True)

In [86]:
fuzz.partial_ratio('hi-tech textile', 'hi tech -textile')

87

---

In [161]:
getBWDuplicates(col="FNC")

{48: ['ivory garments factory l.l.c', 'ivory garments factory'],
 62: ['lee and suy vina', 'l and s vina'],
 178: ['century miracle apparel manufacturing',
  'century miracle-main building'],
 182: ['delta starmark garment', 'concept creator delta garment vietnam'],
 198: ['esquel garment manufacturing vietnam',
  'asia garment manufacturer vietnam'],
 246: ['branch of thuan phuong embroideries garments', 'm.d.k'],
 336: ['yoonlam vina', 'l and t trading produce'],
 364: ['classic fashion apparel industry ltd. co', 'al tafilah apparel'],
 454: ['fashion stitch noah, s.a', 'choi shin nicaragua, s.a'],
 688: ['fashiontex apparel', 'colour apparel garment'],
 704: ['classic fashion apparel industry',
  'standard jeans apparel manufacturing co'],
 747: ['broadland cambodia garment industries',
  'broadland cambodia garment industries . branch 1'],
 809: ['deuck woo', 'deuck woo cambodia'],
 848: ['cg glory manufacturing company limited',
  'ford glory manufacturing limited'],
 880: ['grand

---

In [79]:
bw.FactoryBWID.nunique()

1982

In [78]:
bw.FNC.nunique()

1900

In [77]:
getBWDuplicates(col="FNC")

{48: ['ivory garments factory l.l.c', 'ivory garments factory'],
 62: ['lee and suy vina', 'l and s vina'],
 178: ['century miracle apparel manufacturing',
  'century miracle-main building'],
 182: ['delta starmark garment', 'concept creator delta garment vietnam'],
 198: ['esquel garment manufacturing vietnam',
  'asia garment manufacturer vietnam'],
 246: ['branch of thuan phuong embroideries garments', 'm.d.k'],
 336: ['yoonlam vina', 'l and t trading produce'],
 364: ['classic fashion apparel industry ltd. co', 'al tafilah apparel'],
 454: ['fashion stitch noah, s.a', 'choi shin nicaragua, s.a'],
 688: ['fashiontex apparel', 'colour apparel garment'],
 704: ['classic fashion apparel industry',
  'standard jeans apparel manufacturing co'],
 747: ['broadland cambodia garment industries',
  'broadland cambodia garment industries . branch 1'],
 809: ['deuck woo', 'deuck woo cambodia'],
 848: ['cg glory manufacturing company limited',
  'ford glory manufacturing limited'],
 880: ['grand

- `FNC` &larr;modify- `bw.FactoryName`
- `CNRR` &larr; `ci.company_name`
- `reriskID` &larr; `ci.reriskID`

# 1st round match:

In [104]:
bwFacts1=[strProcess(x,aggresive=1) for x in bwFacts]
ciFacts1=[strProcess(x,aggresive=1) for x in ciFacts]

In [156]:
len(bwFacts1)

2094

In [105]:
match1=list(set(bwFacts1)&set(ciFacts1))

In [114]:
len(match1)

124

In [143]:
len(set(bwFacts1))

1904

# 2nd round match (fuzz, first letter)

In [151]:
fuzz.partial_ratio("content","content sasass")

100

In [117]:
match2={}
for i in tqdm(set(bwFacts1)-set(match1)):
    cand=[x for x in ciFacts1 if x[0]==i[0]]
    possibleMatch=[y for y in cand if fuzz.partial_ratio(i,y)>75]
    if len(possibleMatch)>0:
        match2[i]=possibleMatch
    # sorted(cand,key=lambda x : fuzz.partial_ratio(i,x),reversed=True)

100%|██████████| 1780/1780 [08:22<00:00,  3.54it/s]


In [119]:
len(match2)

971

In [120]:
match2

{'king way enterprises': ['kgn enterprise'],
 'son ha garment jsc': ['song hong garment jsc',
  'son keng garment co ltd',
  'soho sheng he garment co ltd'],
 'golden star corporate development': ['golden star corporate development ltd'],
 'dkdr haiti s.a': ['dkdr haiti sa'],
 'akh apparels ltd': ['asian apparels ltd',
  'align apparels ltd',
  'akik apparels ltd',
  'adiyat apparels ltd',
  'akh eco apparels ltd',
  'amichi apparels ltd',
  'ags apparels ltd',
  'ashulia apparels ltd',
  'armana apparels ltd',
  'agami apparels ltd',
  'apparel art ltd',
  'aurora apparel co ltd',
  'adury apparels ltd',
  'arena apparels ltd',
  'ali apparels ltd',
  'apparel 21 ltd',
  'asif apparels ltd',
  'ats apparels ltd',
  'arunima apparels ltd',
  'anzir apparels ltd',
  'ananta apparels ltd',
  'anowara apparels ltd',
  'ab apparels ltd'],
 'eco way knitwear': ['eco way knitwear co ltd'],
 'seduno investment cambo fashion': ['seduno investment cambo fashion co ltd',
  'saiem fashion'],
 'cg

JSC = "joint.."

In [121]:
{x:match2[x] for x in match2 if len(match2[x])==1}

{'king way enterprises': ['kgn enterprise'],
 'golden star corporate development': ['golden star corporate development ltd'],
 'dkdr haiti s.a': ['dkdr haiti sa'],
 'eco way knitwear': ['eco way knitwear co ltd'],
 'pt doosan sinar sukabumi': ['pt doosan jaya sukabumi'],
 'pt busana indah global': ['pt ba'],
 'heng shing garment manufacturing': ['hung wah garment manufacturing ltd'],
 'viet duc garment branch': ['viet y'],
 'in jae garment': ['international business garments manufacturing company ltd'],
 'roo hsing garment': ['roo hsing garment co ltd'],
 'yorks': ['york fashion ltd'],
 'h and l apparel': ['h and l apparel co ltd'],
 'thien nam garment': ['thien nam garment co ltd'],
 'al mustamerah for tex': ['al'],
 'sinosky hejun garment': ['sinosky hejun garment co ltd'],
 'akh shirts limited (former factory': ['akh shirts ltd'],
 'pt jaya asri garmindo': ['pt ba'],
 'epyllion style ltd': ['epyllion ltd'],
 'pt daese garmin': ['pt ba'],
 'branch of leatherbank': ['bankgo'],
 'xin h

In [129]:
sqldf.run("select distinct(company_name) from ci group by company_name having count(distinct(repriskID))>1")

,company_name
0,Laborosco; PT
1,Lida Textile & Dyeing Ltd
2,Maju Bersama; CV
3,Mujur Timber; PT
4,Nassa Basics Ltd
5,PT Hong Yi Indonesia
6,PT Jaya Bersinar Sejahtera
7,Yasmin Spinning Mills Ltd


In [144]:
bw.columns

Index(['QuestID', 'FactoryBWID', 'FactoryName', 'Country', 'AssessmentDate',
       'year', 'FNC', 'CNRR', 'repriskID'],
      dtype='object')